In [ ]:
%cd "drive/My Drive/darknet"

In [ ]:
%ls

# Detect and save cropped image

#### Utils:

In [ ]:
import os
import glob
import time

import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt


class Cropper:

    def __init__(self, cfg: str, weights: str, thresh: float = 0.5):
        """
        Load model from config and weights file
        """
        self.cfg = cfg
        self.weights = weights
        self.CONFTHRESH = thresh  # set to 0.3 if not working
        self.NMSTHRESH = 0.3
        self.model = cv2.dnn.readNetFromDarknet(self.cfg, self.weights)
        print("- Model ready")

    
    def __show_dif(self, img, crop):
        """
        Returns subplots with original and cropped images
        """
        _, ax = plt.subplots(1, 2, figsize=(8, 4))
        ax[0].set_title('Predicted image')
        ax[0].imshow(img)
        ax[1].set_title('Cropped image')
        ax[1].imshow(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        plt.show()

    def __crop(self, img):
        """
        Get prediction and return cropped image
        """
        (H, W) = img.shape[:2]
        layer_names = self.model.getLayerNames()
        layer_names = [
            layer_names[i[0] - 1] for i in self.model.getUnconnectedOutLayers()
        ]

        blob = cv2.dnn.blobFromImage(
            img,
            1 / 255.0,
            (416, 416),
            swapRB=False,
            crop=False
        )

        self.model.setInput(blob)
        layer_outputs = self.model.forward(layer_names)

        boxes = []
        confidences = []
        classes_ids = []

        for output in tqdm(layer_outputs, desc="- Making prediction"):

            for detected in output:

                scores = detected[5:]
                class_id = np.argmax(scores)
                conf = scores[class_id]

                if conf > self.CONFTHRESH:
                    box = detected[0:4] * np.array([W, H, W, H])
                    (x_center, y_center, width, height) = box.astype("int")

                    x = int(x_center - (width / 2))
                    y = int(y_center - (height / 2))

                    boxes.append(
                        [x, y, int(width), int(height)]
                    )
                    confidences.append(float(conf))
                    classes_ids.append(class_id)

        idxs = cv2.dnn.NMSBoxes(
            boxes,
            confidences,
            self.CONFTHRESH,
            self.NMSTHRESH
        )

        if len(idxs) > 0:

            for i in idxs.flatten():

                (x, y) = (max(0, boxes[i][0]), max(0, boxes[i][1]))
                (w, h) = (boxes[i][2], boxes[i][3])
                cropped_image = img[y:y+h, x:x+w]

        return cropped_image



    def predict(self, item: str):
        """
        Predict one image or a folder of images and save result
        """
        if '.jpg' in item:

            image = plt.imread(item)
            # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            start = time.time()
            output_image = self.__crop(img=image)
            print(f"- Predicted {item} in {time.time() - start:.2f} seconds")

            output_path = os.path.join('predictions/', os.path.basename(item))

            output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

            cv2.imwrite(output_path, output_image)

            print(f"- Saved predicted image to {output_path}")
            self.__show_dif(image, output_image)

        else:

            for i in glob.glob(os.path.join(item, '*.jpg')):

                image = plt.imread(i)
                # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                start = time.time()
                output_image = self.__crop(img=image)
                print(f"- Predicted {i} in {time.time() - start:.2f} seconds")

                output_path = os.path.join('predictions/', os.path.basename(i))

                output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

                cv2.imwrite(output_path, output_image)

                print(f"- Saved predicted image to {output_path}")
                self.__show_dif(image, output_image)

#### Run:

In [ ]:
del crop

In [ ]:
crop = Cropper(
    cfg='cfg/custom/tiny-armoire.cfg',
    weights='backup/tiny-armoire_last.weights',
    thresh=0.5
)

In [ ]:
crop.predict("data/obj/")